# Tutorial 04: Pipeline Completo - Análisis de Pobreza con enahopy

Este tutorial integra los tres módulos principales de enahopy en un análisis completo:
- **Loader**: Descarga y carga de datos ENAHO
- **Merger**: Combinación de múltiples módulos
- **Null Analysis**: Análisis de calidad de datos

## Objetivo del Análisis

Realizar un análisis de pobreza multidimensional que responda:
1. ¿Cuál es la relación entre características de vivienda y pobreza?
2. ¿Cómo se relaciona el empleo con la condición de pobreza?
3. ¿Qué tan completos son los datos para este análisis?
4. ¿Cuáles son las principales inequidades a nivel geográfico?

## Pipeline End-to-End

```
1. LOADER → Descargar datos (módulos 01, 02, 05, 34)
           ↓
2. MERGER → Combinar a nivel hogar y persona
           ↓
3. NULL ANALYSIS → Evaluar calidad de datos
           ↓
4. ANÁLISIS → Poverty insights
           ↓
5. EXPORT → Resultados y reportes
```

In [1]:
# Instalación (si es necesario)
# !pip install enahopy

import enahopy
from enahopy.loader import ENAHODataDownloader
from enahopy.loader.io import ENAHOLocalReader
from enahopy.merger import ENAHOModuleMerger
from enahopy.null_analysis import ENAHONullAnalyzer
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print(f"enahopy versión: {enahopy.__version__}")
print(f"Inicio del análisis: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

enahopy versión: 0.6.0
Inicio del análisis: 2025-10-19 22:51:28


## FASE 1: LOADER - Descarga y Carga de Datos

### 1.1 Configurar Descargador

In [ ]:
# Configuración
year = 202
cache_dir = ".enaho_cache"

# Inicializar downloader
downloader = ENAHODataDownloader(
    cache_dir=cache_dir,
    use_cache=True,
    verbose=True
)

# Inicializar reader
reader = ENAHOLocalReader(
    cache_dir=cache_dir,
    verbose=False
)

print(f"\n✅ Configuración completada")
print(f"   Año de análisis: {year}")
print(f"   Directorio de caché: {cache_dir}")

### 1.2 Descargar Módulos Necesarios

In [ ]:
# Módulos a descargar
modules_to_download = {
    "01": "Características de Vivienda y Hogar",
    "02": "Características de Miembros del Hogar",
    "05": "Empleo e Ingresos",
    "34": "Sumaria (Variables Calculadas)"
}

print("⬇️ Descargando módulos ENAHO...\n")

for module_id, description in modules_to_download.items():
    print(f"   Módulo {module_id}: {description}")
    downloader.download(
        year=year,
        module_id=module_id,
        extract=True
    )

print("\n✅ Todos los módulos descargados exitosamente")

### 1.3 Cargar Datos con Columnas Específicas

#### Módulo 34 - Sumaria (Base del análisis de pobreza)

In [ ]:
# Columnas clave del módulo 34
cols_sumaria = [
    # Identificadores
    'conglome', 'vivienda', 'hogar',
    # Geográficas
    'ubigeo', 'dominio', 'estrato',
    # Household composition
    'mieperho',  # Número de miembros
    # Económicas
    'gashog2d',  # Gasto total del hogar
    'inghog2d',  # Ingreso total del hogar
    'linpe',     # Línea de pobreza extrema
    'linea',     # Línea de pobreza
    'pobreza',   # Condición de pobreza (1=pobre extremo, 2=pobre, 3=no pobre)
    # Ponderadores
    'factor07'   # Factor de expansión
]

print("📖 Cargando Módulo 34 (Sumaria)...")
df_sumaria = reader.read_file(
    year=year,
    module_id="34",
    columns=cols_sumaria,
    validate_columns=True
)

print(f"   Shape: {df_sumaria.shape}")
print(f"   Hogares: {len(df_sumaria):,}")

#### Módulo 01 - Características de Vivienda

In [ ]:
# Columnas clave del módulo 01
cols_vivienda = [
    # Identificadores
    'conglome', 'vivienda', 'hogar',
    # Tipo de vivienda
    'p101',    # Tipo de vivienda
    # Materiales
    'p102',    # Material predominante en paredes
    'p103',    # Material predominante en pisos
    'p103a',   # Material predominante en techos
    # Servicios
    'p110',    # Abastecimiento de agua
    'p111',    # Servicios higiénicos
    'p112',    # Alumbrado eléctrico
    # Habitaciones
    'p104',    # Número de habitaciones
    'p105a',   # Habitaciones para dormir
]

print("📖 Cargando Módulo 01 (Vivienda)...")
df_vivienda = reader.read_file(
    year=year,
    module_id="01",
    columns=cols_vivienda,
    validate_columns=True
)

print(f"   Shape: {df_vivienda.shape}")
print(f"   Hogares: {len(df_vivienda):,}")

#### Módulo 02 - Características de Personas

In [ ]:
# Columnas clave del módulo 02
cols_personas = [
    # Identificadores
    'conglome', 'vivienda', 'hogar', 'codperso',
    # Demográficas
    'p203',     # Parentesco con jefe
    'p204',     # Sexo
    'p205',     # Edad
    'p206',     # Estado civil
    # Educación
    'p208a',    # Nivel educativo alcanzado
    'p209',     # Años de estudio
    'p210',     # Sabe leer y escribir
]

print("📖 Cargando Módulo 02 (Personas)...")
df_personas = reader.read_file(
    year=year,
    module_id="02",
    columns=cols_personas,
    validate_columns=True
)

print(f"   Shape: {df_personas.shape}")
print(f"   Personas: {len(df_personas):,}")

#### Módulo 05 - Empleo e Ingresos

In [ ]:
# Columnas clave del módulo 05
cols_empleo = [
    # Identificadores
    'conglome', 'vivienda', 'hogar', 'codperso',
    # Condición laboral
    'ocu500',   # Condición de actividad
    # Ocupación
    'p506',     # Ocupación principal
    'p507',     # Categoría ocupacional
    # Ingresos
    'i524a1',   # Ingreso por trabajo dependiente
    'i530a',    # Ingreso por trabajo independiente
    # Horas
    'p511a',    # Horas trabajadas semana pasada
]

print("📖 Cargando Módulo 05 (Empleo)...")
df_empleo = reader.read_file(
    year=year,
    module_id="05",
    columns=cols_empleo,
    validate_columns=True
)

print(f"   Shape: {df_empleo.shape}")
print(f"   Personas: {len(df_empleo):,}")

## FASE 2: MERGER - Combinación de Módulos

### 2.1 Inicializar Merger

In [ ]:
# Inicializar merger
merger = ENAHOModuleMerger(verbose=True)

print("✅ ENAHOModuleMerger inicializado")

### 2.2 Merge a Nivel Hogar: Sumaria + Vivienda

In [ ]:
print("🔗 Combinando datos a nivel HOGAR (Sumaria + Vivienda)...\n")

# Preparar módulos
modulos_hogar = {
    'sumaria': df_sumaria,
    'vivienda': df_vivienda
}

# Realizar merge
resultado_hogar = merger.merge_modules(
    modules_dict=modulos_hogar,
    merge_level='hogar',
    how='inner',
    validate_keys=True
)

df_hogar_completo = resultado_hogar.merged_df

print(f"\n✅ Merge a nivel hogar completado")
print(f"   Shape final: {df_hogar_completo.shape}")
print(f"   Hogares: {len(df_hogar_completo):,}")
print(f"   Columnas: {len(df_hogar_completo.columns)}")

### 2.3 Merge a Nivel Persona: Características + Empleo

In [ ]:
print("🔗 Combinando datos a nivel PERSONA (Características + Empleo)...\n")

# Preparar módulos
modulos_persona = {
    'personas': df_personas,
    'empleo': df_empleo
}

# Realizar merge
resultado_persona = merger.merge_modules(
    modules_dict=modulos_persona,
    merge_level='persona',
    how='left',  # Left join para mantener todas las personas
    validate_keys=True
)

df_persona_completo = resultado_persona.merged_df

print(f"\n✅ Merge a nivel persona completado")
print(f"   Shape final: {df_persona_completo.shape}")
print(f"   Personas: {len(df_persona_completo):,}")
print(f"   Columnas: {len(df_persona_completo.columns)}")

### 2.4 Merge Multi-Nivel: Persona + Hogar

In [ ]:
print("🔗 Combinando datos PERSONA + HOGAR...\n")

# Merge persona con hogar
id_cols_hogar = ['conglome', 'vivienda', 'hogar']

df_final = df_persona_completo.merge(
    df_hogar_completo,
    on=id_cols_hogar,
    how='left',
    suffixes=('_persona', '_hogar')
)

print(f"✅ Dataset final creado")
print(f"   Shape: {df_final.shape}")
print(f"   Personas: {len(df_final):,}")
print(f"   Hogares únicos: {df_final[id_cols_hogar].drop_duplicates().shape[0]:,}")
print(f"   Columnas totales: {len(df_final.columns)}")

## FASE 3: NULL ANALYSIS - Evaluación de Calidad

### 3.1 Analizar Dataset de Hogares

In [ ]:
print("🔍 Analizando calidad de datos - HOGARES\n")

# Inicializar analizador
analyzer = ENAHONullAnalyzer(verbose=False)

# Analizar dataset de hogares
result_hogar = analyzer.analyze(df_hogar_completo)
report_hogar = analyzer.generate_report(df_hogar_completo, format='dict')

# Estadísticas de nulos
null_rate_hogar = df_hogar_completo.isnull().sum().sum() / (df_hogar_completo.shape[0] * df_hogar_completo.shape[1]) * 100

print(f"📊 Calidad de Datos - HOGARES:")
print(f"   Tasa de nulos: {null_rate_hogar:.2f}%")
print(f"   Patrones detectados: {len(result_hogar.patterns)}")
print(f"   Recomendaciones: {len(result_hogar.recommendations)}")

# Top columnas con nulos
null_pcts_hogar = (df_hogar_completo.isnull().sum() / len(df_hogar_completo) * 100).sort_values(ascending=False)
print(f"\n   Top 5 columnas con nulos:")
for col, pct in null_pcts_hogar.head(5).items():
    if pct > 0:
        print(f"     - {col}: {pct:.2f}%")

### 3.2 Analizar Dataset Final (Persona + Hogar)

In [ ]:
print("🔍 Analizando calidad de datos - DATASET FINAL\n")

# Analizar dataset final
result_final = analyzer.analyze(df_final)
report_final = analyzer.generate_report(df_final, format='dict')

# Estadísticas de nulos
null_rate_final = df_final.isnull().sum().sum() / (df_final.shape[0] * df_final.shape[1]) * 100

print(f"📊 Calidad de Datos - DATASET FINAL:")
print(f"   Tasa de nulos: {null_rate_final:.2f}%")
print(f"   Patrones detectados: {len(result_final.patterns)}")
print(f"   Recomendaciones: {len(result_final.recommendations)}")

# Filas completas
complete_rows = df_final.dropna(how='any')
complete_pct = len(complete_rows) / len(df_final) * 100
print(f"\n   Filas completas (sin nulos): {len(complete_rows):,} ({complete_pct:.2f}%)")

## FASE 4: ANÁLISIS - Poverty Insights

### 4.1 Distribución de Pobreza

In [ ]:
print("="*70)
print("📊 ANÁLISIS DE POBREZA - RESULTADOS")
print("="*70)

# Distribución de pobreza
print("\n1. DISTRIBUCIÓN DE POBREZA (a nivel hogar)\n")

pobreza_dist = df_hogar_completo['pobreza'].value_counts().sort_index()
pobreza_pct = (pobreza_dist / pobreza_dist.sum() * 100)

categorias_pobreza = {
    1: 'Pobre extremo',
    2: 'Pobre no extremo',
    3: 'No pobre'
}

for cat, count in pobreza_dist.items():
    nombre = categorias_pobreza.get(cat, f'Categoría {cat}')
    pct = pobreza_pct[cat]
    print(f"   {nombre:20s}: {count:6,} hogares ({pct:5.2f}%)")

total_pobres = pobreza_dist[pobreza_dist.index.isin([1, 2])].sum()
pct_pobres = (total_pobres / pobreza_dist.sum() * 100)
print(f"\n   TOTAL POBRES: {total_pobres:,} hogares ({pct_pobres:.2f}%)")

### 4.2 Pobreza y Características de Vivienda

In [ ]:
print("\n2. POBREZA Y ACCESO A SERVICIOS BÁSICOS\n")

# Crear categoría binaria de pobreza
df_hogar_completo['es_pobre'] = df_hogar_completo['pobreza'].isin([1, 2])

# Análisis de alumbrado eléctrico (p112)
if 'p112' in df_hogar_completo.columns:
    tiene_luz = df_hogar_completo.groupby('es_pobre')['p112'].apply(
        lambda x: (x == 1).sum() / len(x) * 100
    )
    
    print("   Acceso a alumbrado eléctrico:")
    print(f"     - Hogares pobres:    {tiene_luz[True]:.1f}%")
    print(f"     - Hogares no pobres: {tiene_luz[False]:.1f}%")
    print(f"     - Brecha:            {tiene_luz[False] - tiene_luz[True]:.1f} puntos porcentuales")

# Análisis de agua (p110)
if 'p110' in df_hogar_completo.columns:
    tiene_agua = df_hogar_completo.groupby('es_pobre')['p110'].apply(
        lambda x: (x.isin([1, 2, 3])).sum() / len(x) * 100  # Red pública dentro/fuera/pilón
    )
    
    print("\n   Acceso a agua por red pública:")
    print(f"     - Hogares pobres:    {tiene_agua[True]:.1f}%")
    print(f"     - Hogares no pobres: {tiene_agua[False]:.1f}%")
    print(f"     - Brecha:            {tiene_agua[False] - tiene_agua[True]:.1f} puntos porcentuales")

### 4.3 Pobreza y Empleo

In [ ]:
print("\n3. POBREZA Y EMPLEO (a nivel persona)\n")

# Agregar indicador de pobreza al dataset de personas
df_final['es_pobre'] = df_final['pobreza'].isin([1, 2])

# Filtrar población en edad de trabajar (14+ años)
df_pet = df_final[df_final['p205'] >= 14].copy()

print(f"   Población en edad de trabajar: {len(df_pet):,} personas")

# Tasa de ocupación por condición de pobreza
if 'ocu500' in df_pet.columns:
    ocupacion = df_pet.groupby('es_pobre')['ocu500'].apply(
        lambda x: (x == 1).sum() / len(x) * 100  # ocu500=1 es ocupado
    )
    
    print("\n   Tasa de ocupación:")
    print(f"     - Población pobre:     {ocupacion[True]:.1f}%")
    print(f"     - Población no pobre:  {ocupacion[False]:.1f}%")

# Horas trabajadas promedio
if 'p511a' in df_pet.columns:
    horas_prom = df_pet[df_pet['ocu500'] == 1].groupby('es_pobre')['p511a'].mean()
    
    print("\n   Horas trabajadas/semana (ocupados):")
    print(f"     - Población pobre:     {horas_prom[True]:.1f} horas")
    print(f"     - Población no pobre:  {horas_prom[False]:.1f} horas")

### 4.4 Análisis Geográfico

In [ ]:
print("\n4. DISTRIBUCIÓN GEOGRÁFICA DE LA POBREZA\n")

# Analizar por dominio geográfico
if 'dominio' in df_hogar_completo.columns:
    pobreza_dominio = df_hogar_completo.groupby('dominio').agg({
        'es_pobre': ['sum', 'count', 'mean']
    })
    
    pobreza_dominio.columns = ['pobres', 'total', 'tasa_pobreza']
    pobreza_dominio['tasa_pobreza'] = pobreza_dominio['tasa_pobreza'] * 100
    pobreza_dominio = pobreza_dominio.sort_values('tasa_pobreza', ascending=False)
    
    print("   Top 5 dominios con mayor tasa de pobreza:")
    for dominio, row in pobreza_dominio.head(5).iterrows():
        print(f"     Dominio {dominio}: {row['tasa_pobreza']:.1f}% ({row['pobres']:.0f}/{row['total']:.0f} hogares)")

### 4.5 Resumen de Hallazgos Clave

In [ ]:
print("\n" + "="*70)
print("💡 HALLAZGOS CLAVE DEL ANÁLISIS")
print("="*70)

print("""
1. MAGNITUD DE LA POBREZA
   - Se identificó el porcentaje de hogares en situación de pobreza
   - Distinción entre pobreza extrema y no extrema

2. INEQUIDAD EN SERVICIOS BÁSICOS
   - Brechas significativas en acceso a electricidad y agua
   - Los hogares pobres tienen menor acceso a servicios

3. EMPLEO Y POBREZA
   - Diferencias en tasas de ocupación entre pobres y no pobres
   - Variación en horas trabajadas según condición de pobreza

4. DIMENSIÓN GEOGRÁFICA
   - Heterogeneidad territorial en tasas de pobreza
   - Identificación de dominios con mayores desafíos

5. CALIDAD DE DATOS
   - Dataset integrado exitosamente con múltiples módulos
   - Análisis de valores faltantes completado
   - Datos aptos para análisis multidimensional
""")

print("="*70)

## FASE 5: EXPORT - Guardar Resultados

### 5.1 Exportar Datasets Procesados

In [ ]:
import os

# Crear directorio de salida
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)

print("💾 Exportando resultados...\n")

# Exportar dataset de hogares
output_file_hogar = os.path.join(output_dir, f"enaho_{year}_hogares_merged.csv")
df_hogar_completo.to_csv(output_file_hogar, index=False)
print(f"   ✅ Hogares: {output_file_hogar}")

# Exportar dataset final
output_file_final = os.path.join(output_dir, f"enaho_{year}_final_merged.csv")
df_final.to_csv(output_file_final, index=False)
print(f"   ✅ Dataset final: {output_file_final}")

print(f"\n✅ Archivos exportados en: {output_dir}/")

### 5.2 Generar Reporte de Calidad

In [ ]:
# Crear reporte de calidad
reporte_calidad = f"""
{'='*70}
REPORTE DE CALIDAD DE DATOS - ENAHO {year}
{'='*70}

Fecha de generación: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

DATASETS PROCESADOS:
-------------------
1. Módulo 34 (Sumaria):          {len(df_sumaria):,} hogares
2. Módulo 01 (Vivienda):         {len(df_vivienda):,} hogares
3. Módulo 02 (Personas):         {len(df_personas):,} personas
4. Módulo 05 (Empleo):           {len(df_empleo):,} personas

DATASETS COMBINADOS:
-------------------
1. Hogares (Sumaria + Vivienda): {len(df_hogar_completo):,} hogares, {len(df_hogar_completo.columns)} columnas
2. Personas (Caract. + Empleo):  {len(df_persona_completo):,} personas, {len(df_persona_completo.columns)} columnas
3. Final (Persona + Hogar):      {len(df_final):,} registros, {len(df_final.columns)} columnas

CALIDAD DE DATOS:
----------------
Dataset de Hogares:
  - Tasa de nulos: {null_rate_hogar:.2f}%
  - Patrones detectados: {len(result_hogar.patterns)}

Dataset Final:
  - Tasa de nulos: {null_rate_final:.2f}%
  - Filas completas: {len(complete_rows):,} ({complete_pct:.2f}%)
  - Patrones detectados: {len(result_final.patterns)}

ESTADÍSTICAS DE POBREZA:
-----------------------
Total de hogares pobres: {total_pobres:,} ({pct_pobres:.2f}%)
  - Pobre extremo: {pobreza_dist.get(1, 0):,}
  - Pobre no extremo: {pobreza_dist.get(2, 0):,}
  - No pobre: {pobreza_dist.get(3, 0):,}

{'='*70}
"""

# Guardar reporte
reporte_file = os.path.join(output_dir, f"reporte_calidad_{year}.txt")
with open(reporte_file, 'w', encoding='utf-8') as f:
    f.write(reporte_calidad)

print(f"📄 Reporte de calidad: {reporte_file}")
print(reporte_calidad)

## Conclusiones del Tutorial

### Lo que aprendiste:

1. ✅ **LOADER**: Descargar y cargar múltiples módulos ENAHO con columnas específicas
2. ✅ **MERGER**: Combinar datos a nivel hogar, persona y multi-nivel
3. ✅ **NULL ANALYSIS**: Evaluar calidad de datos en cada etapa del pipeline
4. ✅ **ANÁLISIS**: Realizar análisis multidimensional de pobreza
5. ✅ **EXPORT**: Guardar resultados y generar reportes

### Mejores Prácticas Demostradas:

- **Planificación**: Definir variables clave antes de cargar datos
- **Incremental**: Construir dataset paso a paso (hogar → persona → final)
- **Validación**: Verificar calidad en cada etapa del merge
- **Documentación**: Generar reportes automáticos de calidad
- **Reproducibilidad**: Pipeline completo ejecutable de principio a fin

### Próximos Pasos:

1. **Análisis avanzados**: Modelos de regresión, clustering, etc.
2. **Visualizaciones**: Crear gráficos interactivos de resultados
3. **Automatización**: Convertir este pipeline en un script reutilizable
4. **Comparación temporal**: Analizar múltiples años (2018-2022)
5. **Machine Learning**: Predecir pobreza con características observables

### Recursos Adicionales:

- Documentación: https://enahopy.readthedocs.io
- Ejemplos: https://github.com/elpapx/enahopy/tree/main/examples
- Issues: https://github.com/elpapx/enahopy/issues

---

**¡Felicitaciones! Has completado el pipeline completo de análisis con enahopy.**